In [1]:
# Import library
import random
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import numpy as np
import spacy
from utils import process_sentence, bleu, save_checkpoint, load_checkpoint
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
from torchtext.data import Field, BucketIterator, TabularDataset, Dataset
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("./data/fra-eng.csv", sep="\t" , header=0, names=["InputText", "OutputText"])

In [3]:
df.tail()

,InputText,OutputText
135836,Une empreinte carbone est la somme de pollutio...,A carbon footprint is the amount of carbon dio...
135837,La mort est une chose qu'on nous décourage sou...,Death is something that we're often discourage...
135838,Puisqu'il y a de multiples sites web sur chaqu...,Since there are usually multiple websites on a...
135839,Si quelqu'un qui ne connaît pas vos antécédent...,If someone who doesn't know your background sa...
135840,Il est peut-être impossible d'obtenir un Corpu...,It may be impossible to get a completely error...


In [4]:
train, valid = train_test_split(df, test_size=0.4)
valid, test = train_test_split(valid, test_size=0.5)

In [5]:
train.to_json("./data/train.json", orient="records", lines=True)
valid.to_json("./data/valid.json", orient="records", lines=True)
test.to_json("./data/test.json", orient="records", lines=True)

In [6]:
spacy_input = spacy.load("fr")
spacy_output = spacy.load("en")

def tokenize_input(text):
    return [token.text for token in spacy_input.tokenizer(text)]

def tokenize_output(text):
    return [token.text for token in spacy_output.tokenizer(text)]

In [7]:
inputText = Field(init_token="<sos>", eos_token="<eos>", tokenize=tokenize_input, lower=True)
outputText = Field(init_token="<sos>", eos_token="<eos>", tokenize=tokenize_output, lower=True)

In [8]:
fields = {"InputText": ("inputText", inputText), "OutputText": ("outputText", outputText)}

In [9]:
train_data, valid_data = TabularDataset.splits(
    path="data", train="train.json", test="valid.json", format="json", fields=fields
)

test_data = TabularDataset.splits(
    path="data", test="test.json", format="json", fields=fields
)

In [10]:
inputText.build_vocab(train_data, max_size=10000, min_freq=2)
outputText.build_vocab(train_data, max_size=10000, min_freq=2)

In [11]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, inputText):
        src_mask = inputText.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, inputText, outputText):
        src_seq_length, N = inputText.shape
        trg_seq_length, N = outputText.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(inputText) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(outputText) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(inputText)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
        )
        out = self.fc_out(out)
        return out

In [12]:
# We're ready to define everything we need for training our Seq2Seq model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load_model = False
save_model = True

# Training hyperparameters
num_epochs = 1000
learning_rate = 3e-4
batch_size = 32

# Model hyperparameters
src_vocab_size = len(inputText.vocab)
trg_vocab_size = len(outputText.vocab)
embedding_size = 512    # default: 512
num_heads = 8
num_encoder_layers = 3  # 6 in paper
num_decoder_layers = 3
dropout = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = inputText.vocab.stoi["<pad>"]

# Tensorboard to get nice loss plot
writer = SummaryWriter("runs/loss_plot")
step = 0

In [21]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.inputText),
    device=device,
)

In [22]:
# batch = next(iter(train_iterator))
# print(batch.input)

In [23]:
model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device)

In [24]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)

In [26]:
pad_idx = inputText.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

In [27]:
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)

sentence = "Qu’est-ce que vous aimez faire pendant votre temps libre?"  # french
# sentence = "ein pferd geht unter einer brücke neben einem boot."    # german
# sentence = "A horse is standing under a bridge beside a boat"

In [28]:
for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

    model.eval()
    processed_sentence = process_sentence(
        model, sentence, inputText, outputText, device, max_length=10
    )

    print(f"Predicted sentence: \n {processed_sentence}")
    model.train()
    losses = []

    for batch_idx, batch in enumerate(train_iterator):
        # Get input and targets and get to cuda
        inp_data = batch.inputText.to(device)
        target = batch.outputText.to(device)

        # Forward prop
        output = model(inp_data, target[:-1, :])

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin.
        # Let's also remove the start token while we're at it
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()

        loss = criterion(output, target)
        losses.append(loss.item())

        # Back prop
        loss.backward()
        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # plot to tensorboard
        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1

    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)

[Epoch 0 / 1000]
=> Saving checkpoint
Lemmatize example sentence: 
 ['fear', 'reply', 'court', 'chubby', 'chubby', 'resembles', 'memory', 'grinning', 'gift', 'resembles']
[Epoch 1 / 1000]
=> Saving checkpoint
Lemmatize example sentence: 
 ['would', 'you', 'like', 'you', 'like', 'to', 'do', 'your', 'time', '?']
[Epoch 2 / 1000]
=> Saving checkpoint
Lemmatize example sentence: 
 ['would', 'you', '<unk>', 'you', 'like', 'to', 'do', 'your', 'time', 'time']
[Epoch 3 / 1000]
=> Saving checkpoint
Lemmatize example sentence: 
 ['would', '<unk>', '<unk>', 'you', 'like', 'to', 'do', 'your', 'free', 'time']
[Epoch 4 / 1000]
=> Saving checkpoint
Lemmatize example sentence: 
 ['will', '<unk>', 'think', 'you', 'like', 'to', 'do', 'your', 'free', 'time']
[Epoch 5 / 1000]
=> Saving checkpoint
Lemmatize example sentence: 
 ['would', 'you', '<unk>', 'that', 'you', 'love', 'your', 'time', 'free', 'time']
[Epoch 6 / 1000]
=> Saving checkpoint
Lemmatize example sentence: 
 ['<unk>', 'you', 'only', 'you', '

KeyboardInterrupt: 